## Examples taken from https://docs.python.org/3/library/asyncio-task.html#awaitables

In [4]:
import asyncio
import sys
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

async def say_after(delay, what):
    await asyncio.sleep(delay)
    logger.debug(what)

async def main():
    logger.debug('Started')
    
    await say_after(1, 'hello')
    await say_after(2, 'world')
    
    logger.debug('Finished')

await main() 

[D] Started
[D] hello
[D] world
[D] Finished


In [6]:
## io-bound 
import asyncio
import sys
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks():
    logger.debug(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    # for t in asyncio.all_tasks():
    #     logger.debug(t)
    #     logger.debug('===============================')

async def say_after(delay, what):
    logger.debug(f'Waiting {what} for {delay} sec')
    await asyncio.sleep(delay)
    logger.debug(what)

async def main():
    logger.debug('Started')
    print_all_tasks()
    
    task1 = asyncio.create_task(
        say_after(1, 'hello'))
    print_all_tasks()
    
    task2 = asyncio.create_task(
        say_after(2, 'world'))
    print_all_tasks()
    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    logger.debug('Waiting task1')
    await task1
    
    print_all_tasks()
    logger.debug('Waiting task2')
    await task2
    
    print_all_tasks()
    logger.debug('Finished')

await main()

[D] Started
[D] ========== 1 Tasks ==========
[D] ========== 2 Tasks ==========
[D] ========== 3 Tasks ==========
[D] Waiting task1
[D] Waiting hello for 1 sec
[D] Waiting world for 2 sec
[D] hello
[D] ========== 2 Tasks ==========
[D] Waiting task2
[D] world
[D] ========== 1 Tasks ==========
[D] Finished


## [Tasks](https://docs.python.org/3/library/asyncio-task.html#awaitables)

Tasks are used to schedule coroutines concurrently.

When a coroutine is wrapped into a Task with functions like asyncio.create_task() the coroutine is automatically scheduled to run soon.

When execute asyncio.create_task(), a task is added in the running event loop, when encounter keyword await, the current execution will be stopped and the control will be switched to the next process until the previous process is done.

In [9]:
# cpu-bound
import asyncio
import sys
import time
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks():
    logger.debug(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    # for t in asyncio.all_tasks():
    #     logger.debug(t)
    #     logger.debug('===============================')

async def say_after(delay, what):
    logger.debug(f'Waiting {what} for {delay} sec')
    time.sleep(delay)
    logger.debug(what)

async def main():
    logger.debug('Started')
    print_all_tasks()
    
    task1 = asyncio.create_task(
        say_after(1, 'hello'))
    print_all_tasks()
    
    task2 = asyncio.create_task(
        say_after(2, 'world'))
    print_all_tasks()
    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    logger.debug('Waiting task1')
    await task1
    
    print_all_tasks()
    logger.debug('Waiting task2')
    await task2
    
    print_all_tasks()
    logger.debug('Finished')

await main()

[D] Started
[D] ========== 1 Tasks ==========
[D] ========== 2 Tasks ==========
[D] ========== 3 Tasks ==========
[D] Waiting task1
[D] Waiting hello for 1 sec
[D] hello
[D] Waiting world for 2 sec
[D] world
[D] ========== 1 Tasks ==========
[D] Waiting task2
[D] ========== 1 Tasks ==========
[D] Finished


Coroutine is blocked by time.sleep() which is a simulated cpu-bound execution

In [48]:
# gather 1
import asyncio
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    return f

async def main():
    # Schedule three calls *concurrently*:
    L = asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    logger.debug(L)
    await L
    logger.debug(L)

await main()

MainThread - 2021-06-19 19:31:18,510 - root - DEBUG - <_GatheringFuture pending>
MainThread - 2021-06-19 19:31:18,511 - root - DEBUG - Task A: Compute factorial(2), currently i=2...
MainThread - 2021-06-19 19:31:18,511 - root - DEBUG - Task B: Compute factorial(3), currently i=2...
MainThread - 2021-06-19 19:31:18,512 - root - DEBUG - Task C: Compute factorial(4), currently i=2...
MainThread - 2021-06-19 19:31:19,516 - root - DEBUG - Task A: factorial(2) = 2
MainThread - 2021-06-19 19:31:19,517 - root - DEBUG - Task B: Compute factorial(3), currently i=3...
MainThread - 2021-06-19 19:31:19,519 - root - DEBUG - Task C: Compute factorial(4), currently i=3...
MainThread - 2021-06-19 19:31:20,521 - root - DEBUG - Task B: factorial(3) = 6
MainThread - 2021-06-19 19:31:20,522 - root - DEBUG - Task C: Compute factorial(4), currently i=4...
MainThread - 2021-06-19 19:31:21,525 - root - DEBUG - Task C: factorial(4) = 24
MainThread - 2021-06-19 19:31:21,527 - root - DEBUG - <_GatheringFuture fin

In [11]:
# gather 2
import asyncio
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    logger.info(f"Task {name} is finished.") # Expect CancelledError
    print_all_tasks()
    return f

async def main():
    logger.debug('Started scheduling')
    print_all_tasks()
    task1 = asyncio.create_task(factorial("A", 2))
    task2 = asyncio.create_task(factorial("B", 3))
    task3 = asyncio.create_task(factorial("C", 4))
    logger.debug('Finished scheduling')
    print_all_tasks()
    L = await asyncio.gather(task1, task2, task3)
    print_all_tasks()
    logger.debug(L)

await main()

2021-06-19 20:08:10 | DEBUG    | root | Started scheduling
2021-06-19 20:08:10 | INFO     | root | ========== 1 Tasks ==========
2021-06-19 20:08:10 | DEBUG    | root | Finished scheduling
2021-06-19 20:08:10 | INFO     | root | ========== 4 Tasks ==========
2021-06-19 20:08:10 | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-19 20:08:10 | DEBUG    | root | Task B: Compute factorial(3), currently i=2...
2021-06-19 20:08:10 | DEBUG    | root | Task C: Compute factorial(4), currently i=2...
2021-06-19 20:08:11 | DEBUG    | root | Task A: factorial(2) = 2
2021-06-19 20:08:11 | INFO     | root | Task A is finished.
2021-06-19 20:08:11 | INFO     | root | ========== 4 Tasks ==========
2021-06-19 20:08:11 | DEBUG    | root | Task B: Compute factorial(3), currently i=3...
2021-06-19 20:08:11 | DEBUG    | root | Task C: Compute factorial(4), currently i=3...
2021-06-19 20:08:12 | DEBUG    | root | Task B: factorial(3) = 6
2021-06-19 20:08:12 | INFO     | root | Task B

*awaitable* asyncio.**gather**(*aws, loop=None, return_exceptions=False)  
- Run awaitable objects in the aws sequence concurrently.  
- If any awaitable in aws is a coroutine, it is automatically scheduled as a Task.

Above two examples show that asyncio.**gather**() function accepts both coroutine functions or tasks.

When corountine functions return, that specific task is removed automatically.

In [12]:
# cancel gather 1
import asyncio
from asyncio import CancelledError
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

async def factorial(name, number):
    f = 1
    global L
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    
    # Here, a task not really finish because it still not return yet
    logger.info(f"Task {name} is finished. Trying to cancel remaining tasks") # Expect CancelledError
    
    print_all_tasks(detail=True)
    
    # Here, because the tasks still not return, it turns out all 3 tasks are cancelled.
    L.cancel()
    return f

L = None
async def main():
    global L
    # Schedule three calls *concurrently*:
    L = asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    logger.debug(L)
    try:
        await L
    except CancelledError as ce:
        logger.warning(L)
    
    print_all_tasks(detail=True)

await main()

2021-06-19 20:11:04 | DEBUG    | root | <_GatheringFuture pending>
2021-06-19 20:11:04 | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-19 20:11:04 | DEBUG    | root | Task B: Compute factorial(3), currently i=2...
2021-06-19 20:11:04 | DEBUG    | root | Task C: Compute factorial(4), currently i=2...
2021-06-19 20:11:05 | DEBUG    | root | Task A: factorial(2) = 2
2021-06-19 20:11:05 | INFO     | root | Task A is finished. Trying to cancel remaining tasks
2021-06-19 20:11:05 | INFO     | root | ========== 4 Tasks ==========
2021-06-19 20:11:05 | INFO     | root | <Task pending name='Task-42' coro=<InteractiveShell.run_cell_async() running at /Users/jack/Documents/Concurrency/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169> wait_for=<_GatheringFuture pending cb=[<TaskWakeupMethWrapper object at 0x116eb3d90>()]> cb=[IPythonKernel._cancel_on_sigint.<locals>.cancel_unless_done(<Future pendi...ernel.py:226]>)() at /Users/jack/Documents/Concur

In this example, **Task A** is actually not finished because all 3 tasks including **Task A** are cancelled before **Task A** is returned.

[Some **important** issue about asyncio.**gather**](https://stackoverflow.com/a/59074112)

In [23]:
# cancel gather 2
import asyncio
from asyncio import CancelledError
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

async def cancel_tasks():
    for task in asyncio.all_tasks():
        task.cancel()

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    print_all_tasks()
    return f

async def main():
    print_all_tasks()
    task1 = asyncio.create_task(factorial("A", 2))
    task0 = asyncio.create_task(cancel_tasks())
    task2 = asyncio.create_task(factorial("B", 3))
    task3 = asyncio.create_task(factorial("C", 4))
    print_all_tasks()
    L = asyncio.gather(task1, task0, task2, task3)
    try:
        await L
        logger.debug(L)
    except CancelledError as ce:
        logger.debug(L)
        logger.info('Remaining tasks are canclled')

await main()

2021-06-19 20:29:05 | INFO     | root | ========== 1 Tasks ==========
2021-06-19 20:29:05 | INFO     | root | ========== 5 Tasks ==========
2021-06-19 20:29:05 | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-19 20:29:05 | DEBUG    | root | <_GatheringFuture finished exception=CancelledError()>
2021-06-19 20:29:05 | INFO     | root | Remaining tasks are canclled


In this example, unlike previous example (**Task A** is at least finished computational step and waiting for return), no task finished computational step because when **Task A** *await*, the control is switched to another execution which is cancel_tasks() function.

In [35]:
# cancel gather 2
import asyncio
from asyncio import CancelledError
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

def cancel_tasks(fut):
    logger.info(f'Received a future with value: {fut.result()}')
    logger.info('Remaining Tasks')
    print_all_tasks()
    for task in asyncio.all_tasks():
        task.cancel()

async def factorial(name, number):
    global L
    logger.debug(L)
    f = 1
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    return f

L = None
async def main():
    global L
    logger.debug('Started scheduling')
    print_all_tasks()
    
    task1 = asyncio.create_task(factorial("A", 2))
    task2 = asyncio.create_task(factorial("B", 3))
    task3 = asyncio.create_task(factorial("C", 4))
    task1.add_done_callback(cancel_tasks)
    
    logger.debug('Finished scheduling')
    print_all_tasks()

    L = asyncio.gather(task1, task2, task3)
    try:
        logger.debug(L)
        await L
    except CancelledError as ce:
        L.done()
        logger.debug(L)
        logger.info('Remaining tasks are canclled')
    logger.debug(L)

await main()

2021-06-19 20:43:23 | DEBUG    | root | Started scheduling
2021-06-19 20:43:23 | INFO     | root | ========== 1 Tasks ==========
2021-06-19 20:43:23 | DEBUG    | root | Finished scheduling
2021-06-19 20:43:23 | INFO     | root | ========== 4 Tasks ==========
2021-06-19 20:43:23 | DEBUG    | root | <_GatheringFuture pending>
2021-06-19 20:43:23 | DEBUG    | root | <_GatheringFuture pending cb=[<TaskWakeupMethWrapper object at 0x1176cae20>()]>
2021-06-19 20:43:23 | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-19 20:43:23 | DEBUG    | root | <_GatheringFuture pending cb=[<TaskWakeupMethWrapper object at 0x1176cae20>()]>
2021-06-19 20:43:23 | DEBUG    | root | Task B: Compute factorial(3), currently i=2...
2021-06-19 20:43:23 | DEBUG    | root | <_GatheringFuture pending cb=[<TaskWakeupMethWrapper object at 0x1176cae20>()]>
2021-06-19 20:43:23 | DEBUG    | root | Task C: Compute factorial(4), currently i=2...
2021-06-19 20:43:24 | DEBUG    | root | Task A: facto